In [52]:
import numpy as np
import math

## 1D convolution for text

In [2]:
text = np.array([
    [.2, .1, -.3, .4],
    [.5, .2, -.3, -.1],
    [-.1, -.3, -.2, .4],
    [.3, -.3, .1, .1],
    [.2, -.3, .4, .2],
    [.1, .2, -.1, -.1],
    [-.4, -.4, .2, .3]
])

In [4]:
kernel = np.array([
    [3, 1, 2, -3],
    [-1, 2, 1, -3],
    [1, 1, -1, 1]
])

In [25]:
def get_conv(t, ks):
    return [[np.around(np.sum(t[i:i+3, :] * k), decimals=1) 
            for k in ks] 
            for i in range(t.shape[0] - 2)
            ]

In [26]:
get_conv(text, [kernel])

[[-1.0], [-0.5], [-3.6], [-0.2], [0.3]]

## 1D convolution with padding

In [15]:
text_pad = np.zeros((text.shape[0] + 2, text.shape[1]))

In [16]:
text.shape, text_pad.shape

((7, 4), (9, 4))

In [18]:
text_pad[1:8, :] = text.copy()

In [19]:
text_pad

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 0.2,  0.1, -0.3,  0.4],
       [ 0.5,  0.2, -0.3, -0.1],
       [-0.1, -0.3, -0.2,  0.4],
       [ 0.3, -0.3,  0.1,  0.1],
       [ 0.2, -0.3,  0.4,  0.2],
       [ 0.1,  0.2, -0.1, -0.1],
       [-0.4, -0.4,  0.2,  0.3],
       [ 0. ,  0. ,  0. ,  0. ]])

In [27]:
get_conv(text_pad, [kernel])

[[-0.6], [-1.0], [-0.5], [-3.6], [-0.2], [0.3], [-0.5]]

## 3 channel 1D convolution with padding

In [28]:
kernels = np.array([
    [[3, 1, 2, -3], [-1, 2, 1, -3], [1, 1, -1, 1]],
    [[1, 0, 0, 1], [1, 0, -1, -1], [0, 1, 0, 1]],
    [[1, -1, 2, -1], [1, 0, -1, 3], [0, 2, 2, 1]]
])

In [31]:
conv_1D3C = get_conv(text_pad, kernels)

In [32]:
conv_1D3C

[[-0.6, 0.2, 1.4],
 [-1.0, 1.6, -1.0],
 [-0.5, -0.1, 0.8],
 [-3.6, 0.3, 0.3],
 [-0.2, 0.1, 1.2],
 [0.3, 0.6, 0.9],
 [-0.5, -0.9, 0.1]]

## 3 channel 1D convolution with padding and pooling (through time)

### max-pooling 

In [34]:
np.max(conv_1D3C, axis=0)

array([0.3, 1.6, 1.4])

### average-pooling 

In [36]:
np.around(np.mean(conv_1D3C, axis=0), decimals=2)

array([-0.87,  0.26,  0.53])

## other less useful notions 

### stride

In [44]:
def get_conv_stride(t, ks, stride=1):
    return [[np.around(np.sum(t[i:i+3, :] * k), decimals=1) 
            for k in ks] 
            for i in range(0, t.shape[0] - 2, stride)
            ]

In [46]:
get_conv_stride(text_pad, kernels, stride=2)

[[-0.6, 0.2, 1.4], [-0.5, -0.1, 0.8], [-0.2, 0.1, 1.2], [-0.5, -0.9, 0.1]]

### local pooling

In [68]:
def get_local_pooling(t, stride=2):
    return np.array([np.max(t[i:i+stride], axis=0) for i in range(0, t.shape[0], stride)])

In [61]:
conv_1D3C = np.array(conv_1D3C)

In [62]:
conv_1D3C_pad = np.zeros((conv_1D3C.shape[0]+1, conv_1D3C.shape[1]))

In [63]:
conv_1D3C_pad.shape, conv_1D3C.shape

((8, 3), (7, 3))

In [64]:
conv_1D3C_pad[:conv_1D3C.shape[0], :] = conv_1D3C.copy()

In [65]:
conv_1D3C_pad[conv_1D3C.shape[0]] = -math.inf

In [66]:
conv_1D3C_pad

array([[-0.6,  0.2,  1.4],
       [-1. ,  1.6, -1. ],
       [-0.5, -0.1,  0.8],
       [-3.6,  0.3,  0.3],
       [-0.2,  0.1,  1.2],
       [ 0.3,  0.6,  0.9],
       [-0.5, -0.9,  0.1],
       [-inf, -inf, -inf]])

In [69]:
get_local_pooling(conv_1D3C_pad)

array([[-0.6,  1.6,  1.4],
       [-0.5,  0.3,  0.8],
       [ 0.3,  0.6,  1.2],
       [-0.5, -0.9,  0.1]])

### k-max pooling 

In [70]:
x = np.array([-.6, -1., -.5, -3.6, -.2, .3, -.5])

In [76]:
np.sort(x)[-2:]

array([-0.2,  0.3])

In [77]:
conv_1D3C

array([[-0.6,  0.2,  1.4],
       [-1. ,  1.6, -1. ],
       [-0.5, -0.1,  0.8],
       [-3.6,  0.3,  0.3],
       [-0.2,  0.1,  1.2],
       [ 0.3,  0.6,  0.9],
       [-0.5, -0.9,  0.1]])

In [78]:
np.sort(conv_1D3C, axis=0)

array([[-3.6, -0.9, -1. ],
       [-1. , -0.1,  0.1],
       [-0.6,  0.1,  0.3],
       [-0.5,  0.2,  0.8],
       [-0.5,  0.3,  0.9],
       [-0.2,  0.6,  1.2],
       [ 0.3,  1.6,  1.4]])

In [79]:
np.sort(conv_1D3C, axis=0)[-2:, :]

array([[-0.2,  0.6,  1.2],
       [ 0.3,  1.6,  1.4]])

### dilation

In [80]:
kernels_dilation = np.array([
    [[2, 3, 1], [1, -1, -1], [3, 1, 0]],
    [[1, 3, 1], [1, -1, -1], [3, 1, -1]]
])

In [87]:
kernels_dilation

array([[[ 2,  3,  1],
        [ 1, -1, -1],
        [ 3,  1,  0]],

       [[ 1,  3,  1],
        [ 1, -1, -1],
        [ 3,  1, -1]]])

In [83]:
conv_1D3C

array([[-0.6,  0.2,  1.4],
       [-1. ,  1.6, -1. ],
       [-0.5, -0.1,  0.8],
       [-3.6,  0.3,  0.3],
       [-0.2,  0.1,  1.2],
       [ 0.3,  0.6,  0.9],
       [-0.5, -0.9,  0.1]])

In [109]:
def get_conv_dilation(conv_arr):
    conv_dil = np.array([
        get_conv(conv_arr[[i, i+2, i+4], :], kernels_dilation)
        for i in range(3)
    ])
    return conv_dil.reshape(conv_dil.shape[0], conv_dil.shape[2])

In [110]:
conv_dilation = get_conv_dilation(conv_1D3C)

In [111]:
conv_dilation

array([[-0.9, -1.5],
       [-0.9, -0.8],
       [-4.4, -4. ]])